
1.Find second highest salary

2.Find nth highest salary

3.Find duplicates record

4.Delete duplicates record

5.emp earning more than manager

6.EMP joined before their maneger

7.Get rolling average from last 7 days

8.Running total of sales

9.Top 3 salary per department

10.Find department with no employees

11.Emp with same job in multiple dept

12.EMP who work in more than 1 department

13.Find total salary per department

14.Find highest paid department (by total sales)

15.Find the consecutive days per user -use of lag and lead

16.Calculate monthly revenue

17.Find customers who didn't place any orders

18.Find emp with maximum tenure

19.Rank Department by average salary

20.Find users who purchase the same product twice

21.Emp whose salary < department average

22.Find first and last purchase date for each customer

23.Output rows for two table join - for left/right/full/inner

24.Case statement

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date 

In [0]:
# Orders Table
orders_data = [
    (1, 101, 1001, date(2025, 10, 1), 250.50),
    (2, 102, 1002, date(2025, 10, 2), 450.75),
    (3, 103, 1003, date(2025, 10, 3), 120.00),
    (4, 104, 1002, date(2025, 10, 4), 300.25),
    (5, 105, 1004, date(2025, 10, 5), 550.00),
    (6, 106, 1005, date(2025, 10, 6), 275.75),
    (7, 107, 1001, date(2025, 10, 7), 220.00),
    (8, 108, 1003, date(2025, 10, 8), 180.50)
]

orders_columns = ["order_id", "customer_id", "product_id", "order_date", "amount"]
orders_df = spark.createDataFrame(orders_data, orders_columns)

# Sales Table
sales_data = [
    (1, 1001, date(2025, 10, 5), 260.00),
    (2, 1002, date(2025, 10, 6), 470.50),
    (3, 1003, date(2025, 10, 7), 130.25),
    (4, 1004, date(2025, 10, 8), 560.00),
    (5, 1005, date(2025, 10, 9), 290.00),
    (6, 1006, date(2025, 10, 10), 330.75),
    (7, 1001, date(2025, 10, 11), 240.25),
    (8, 1002, date(2025, 10, 12), 460.00)
]

sales_columns = ["sale_id", "product_id", "sale_date", "amount"]
sales_df = spark.createDataFrame(sales_data, sales_columns)

# Departments Table
departments_data = [
    (10, "Electronics"),
    (20, "Home Appliances"),
    (30, "Furniture"),
    (40, "Groceries"),
    (50, "Clothing")
]

departments_columns = ["dept_id", "dept_name"]
dept_df = spark.createDataFrame(departments_data, departments_columns)

emp_data = [
    (1, "Adam Scott", 10, 3, 55000, "1998-01-10", "Developer"),
    (2, "Bella Rose", 20, 5, 60000, "2021-03-15", "Analyst"),
    (3, "Charlie King", 10, None, 80000, "2019-05-20", "Manager"),
    (4, "Diana Prince", 20, 5, 62000, "2020-07-25", "Team Lead"),
    (5, "Ethan Hunt", 20, None, 90000, "2018-02-10", "Director"),
    (6, "Fiona Glenanne", 30, 7, 50000, "2022-09-05", "QA Engineer"),
    (7, "George Clooney", 30, None, 85000, "2017-11-30", "Manager"),
    (8, "Hannah Baker", 10, 3, 57000, "2021-12-12", "Developer")
]

# Define column names
emp_columns = ["emp_id", "emp_name", "dept_id", "manager_id", "salary", "hire_date", "job_title"]

# Create DataFrame
emp_df = spark.createDataFrame(emp_data, emp_columns)

# Register as a temporary SQL table
emp_df.createOrReplaceTempView("emp_tbl")

# ----------------------------------------------------------
# 3️⃣ Register as Spark SQL Temporary Tables
# ----------------------------------------------------------

orders_df.createOrReplaceTempView("orders_tbl")
sales_df.createOrReplaceTempView("sales_tbl")
dept_df.createOrReplaceTempView("dept_tbl")


###  1. Find second highest salary

In [0]:
# 1.Find second highest salary
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


### a. Using sub-query

In [0]:
spark.sql("""
          select max(salary) as max_salary
          from emp_tbl
          where salary<(
            select max(salary)
            from emp_tbl
          )

          """).display()

max_salary
85000


### b. Using window function

In [0]:
spark.sql("""
          select salary
          from (
              select salary,dense_rank() over(order by salary desc) as rank
              from emp_tbl

          )ranked
          where rank=2
          
          
          """).display()

salary
85000


### c. Using limit

In [0]:
spark.sql("""
          select salary
          from emp_tbl
          order by salary desc
          limit 1 offset 1
          
          
          """).display()

salary
85000


### d. Using limit

In [0]:
spark.sql("""
    SELECT DISTINCT salary
    FROM emp_tbl
    ORDER BY salary DESC
    LIMIT 2
""").display()

salary
90000
85000


### 2. Find nth highest salary

In [0]:
# 2 Find nth highest salary
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


### a. Using CTE

In [0]:
#1
spark.sql("""
          with cte as(
              select *,dense_rank() over(order by salary desc) as rank
              from emp_tbl
          )
          select emp_name,salary from cte
          where rank=3
                    """).display()

emp_name,salary
Charlie King,80000


### 2. Using sub-query

In [0]:
#2
spark.sql("""
          select emp_name,salary
          from (
              select emp_name,salary,dense_rank() over(order by salary desc) as rank
              from emp_tbl
          )ranked
          where rank=3
          
          """).display()

emp_name,salary
Charlie King,80000


### c. Using sub-query

In [0]:
#3
spark.sql("""
    SELECT MIN(salary) AS third_highest_salary
    FROM (
        SELECT DISTINCT salary
        FROM emp_tbl
        ORDER BY salary DESC
        LIMIT 3
    )
""").display()


third_highest_salary
80000


### 3. Find duplicates

In [0]:
# Find duplicates
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
spark.sql("""
          select emp_id,count(*) as count
          from emp_tbl
          group by emp_id
          having count(*)>1
          
          
          """).display()

emp_id,count


### 4. Delete duplicate records

### 5. emp earning more than manager

In [0]:
#emp earning more than manager
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
spark.sql("""
          select e.emp_name,m.emp_name
          from emp_tbl e
          inner join emp_tbl m
          on e.manager_id=m.emp_id
          where e.salary>m.salary

          """).display()

emp_name,emp_name


### 6. EMP joined before their maneger

In [0]:
# EMP joined before their maneger
spark.sql("""
          select e.emp_name,e.hire_date,m.emp_name,m.hire_date
          from emp_tbl e
          inner join emp_tbl m
          on e.manager_id=m.emp_id
          where e.hire_date<m.hire_date
          
          """).display()

emp_name,hire_date,emp_name,hire_date
Adam Scott,1998-01-10,Charlie King,2019-05-20


### 7. Get rolling average from last 7 days

In [0]:
# Get rolling average from last 7 days

In [0]:
sales_df.display()

sale_id,product_id,sale_date,amount
1,1001,2025-10-05,260.0
2,1002,2025-10-06,470.5
3,1003,2025-10-07,130.25
4,1004,2025-10-08,560.0
5,1005,2025-10-09,290.0
6,1006,2025-10-10,330.75
7,1001,2025-10-11,240.25
8,1002,2025-10-12,460.0


### a.

In [0]:
spark.sql("""
          select *,avg(amount) over(order by sale_date rows between 6 preceding and current row) as roll_avg
          from sales_tbl
          """).display()

sale_id,product_id,sale_date,amount,roll_avg
1,1001,2025-10-05,260.0,260.0
2,1002,2025-10-06,470.5,365.25
3,1003,2025-10-07,130.25,286.9166666666667
4,1004,2025-10-08,560.0,355.1875
5,1005,2025-10-09,290.0,342.15
6,1006,2025-10-10,330.75,340.25
7,1001,2025-10-11,240.25,325.9642857142857
8,1002,2025-10-12,460.0,354.5357142857143


### b. Using per product

In [0]:
spark.sql("""
          select *,avg(amount) over(partition by product_id order by sale_date rows between 6 preceding and current row) as roll_avg
          from sales_tbl
          
          """).display()

sale_id,product_id,sale_date,amount,roll_avg
1,1001,2025-10-05,260.0,260.0
7,1001,2025-10-11,240.25,250.125
2,1002,2025-10-06,470.5,470.5
8,1002,2025-10-12,460.0,465.25
3,1003,2025-10-07,130.25,130.25
4,1004,2025-10-08,560.0,560.0
5,1005,2025-10-09,290.0,290.0
6,1006,2025-10-10,330.75,330.75


### 8. Running total of sales

In [0]:
#Running total of sales
spark.sql("""
          select *,sum(amount) over (order by sale_date ) as running_sum
          from sales_tbl
          
          """).display()

sale_id,product_id,sale_date,amount,running_sum
1,1001,2025-10-05,260.0,260.0
2,1002,2025-10-06,470.5,730.5
3,1003,2025-10-07,130.25,860.75
4,1004,2025-10-08,560.0,1420.75
5,1005,2025-10-09,290.0,1710.75
6,1006,2025-10-10,330.75,2041.5
7,1001,2025-10-11,240.25,2281.75
8,1002,2025-10-12,460.0,2741.75


### 9. Top 3 salary per department

In [0]:
#Top 3 salary per department
spark.sql("""
          with cte as(
              select dept_id,salary,dense_rank() over(partition by dept_id order by salary)as rank
              from emp_tbl
          )

          select * 
          from cte
          where rank<4
          """).display()

dept_id,salary,rank
10,55000,1
10,57000,2
10,80000,3
20,60000,1
20,62000,2
20,90000,3
30,50000,1
30,85000,2


### 10. Find department with no employees

In [0]:
dept_df.display()
emp_df.display()

dept_id,dept_name
10,Electronics
20,Home Appliances
30,Furniture
40,Groceries
50,Clothing


emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
#Find department with no employees
spark.sql("""
          select d.*
          from dept_tbl d
          left join emp_tbl e
          on d.dept_id=e.dept_id
          where e.dept_id is null
          
          
          """).display()

dept_id,dept_name
40,Groceries
50,Clothing


### 11. Emp with same job in multiple dept
find employees who have worked in more than one department.

In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
spark.sql("""
          select emp_name,job_title,count(distinct dept_id) as no_of_dept
          from emp_tbl
          group by emp_name,job_title
          having count(distinct dept_id)>1
          """).display()

emp_name,job_title,no_of_dept


### 12. EMP who work in more than 1 department

In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
spark.sql(
    """
    select emp_id,count(distinct dept_id) as dept_cnt
    from emp_tbl
    group by emp_id
    having count(distinct dept_id)>1"""
).display()

emp_id,dept_cnt


### 13. Find total salary per department

In [0]:
spark.sql("""
          select dept_id,sum(salary) as total_salary
          from emp_tbl
          group by dept_id""").display()

dept_id,total_salary
10,192000
20,212000
30,135000


### 14. Find highest paid department (by total sales)

In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
spark.sql("""
          select dept_id,sum(salary) as total_salary
          from emp_tbl
          group by dept_id
          order by total_salary desc
          limit 1
          
          """).display()

dept_id,total_salary
20,212000


### 15. Find the consecutive login days per user (-use of lag and lead)

### 16. Calculate monthly revenue

In [0]:
sales_df.display()

sale_id,product_id,sale_date,amount
1,1001,2025-10-05,260.0
2,1002,2025-10-06,470.5
3,1003,2025-10-07,130.25
4,1004,2025-10-08,560.0
5,1005,2025-10-09,290.0
6,1006,2025-10-10,330.75
7,1001,2025-10-11,240.25
8,1002,2025-10-12,460.0


In [0]:
spark.sql("""
          select date_format(sale_date,'yyyy-MM') as month,
          sum(amount) as monthly_revenue
          from sales_tbl
          group by date_format(sale_date,'yyyy-MM')
          order by month
          """).display()

month,monthly_revenue
2025-10,2741.75


### 17. Find customers who didn't place any orders

In [0]:
orders_df.display()
sales_df.display()

order_id,customer_id,product_id,order_date,amount
1,101,1001,2025-10-01,250.5
2,102,1002,2025-10-02,450.75
3,103,1003,2025-10-03,120.0
4,104,1002,2025-10-04,300.25
5,105,1004,2025-10-05,550.0
6,106,1005,2025-10-06,275.75
7,107,1001,2025-10-07,220.0
8,108,1003,2025-10-08,180.5


sale_id,product_id,sale_date,amount
1,1001,2025-10-05,260.0
2,1002,2025-10-06,470.5
3,1003,2025-10-07,130.25
4,1004,2025-10-08,560.0
5,1005,2025-10-09,290.0
6,1006,2025-10-10,330.75
7,1001,2025-10-11,240.25
8,1002,2025-10-12,460.0


In [0]:
spark.sql("""
    select distinct o.customer_id
    from orders_tbl o
    left join sales_tbl s
    on o.product_id = s.product_id
    where s.product_id is null
""").display()

customer_id


### 18. Find emp with maximum tenure

In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
spark.sql("""
          select emp_name,datediff(day,hire_date,getdate()) as tenure 
          from emp_tbl
          order by tenure desc
          limit 1
          """).display()

emp_name,tenure
Adam Scott,10160


### 19. Rank Department by average salary

In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
spark.sql("""
          select dept_id, avg(salary) as avg_salary 
          from emp_tbl group by dept_id""").display()

dept_id,avg_salary
10,64000.0
20,70666.66666666667
30,67500.0


In [0]:
# Actual answer
spark.sql("""
          select dept_id, avg(salary) as avg_salary, rank() over(order by avg(salary) desc) as rnk 
          from emp_tbl group by dept_id""").display()

dept_id,avg_salary,rnk
20,70666.66666666667,1
30,67500.0,2
10,64000.0,3


### 20. Find users who purchase the same product twice

In [0]:
orders_df.display()

order_id,customer_id,product_id,order_date,amount
1,101,1001,2025-10-01,250.5
2,102,1002,2025-10-02,450.75
3,103,1003,2025-10-03,120.0
4,104,1002,2025-10-04,300.25
5,105,1004,2025-10-05,550.0
6,106,1005,2025-10-06,275.75
7,107,1001,2025-10-07,220.0
8,108,1003,2025-10-08,180.5


In [0]:
spark.sql("""
          select customer_id,product_id 
          from orders_tbl
          group by customer_id,product_id
          having count(*)>1""").display()

customer_id,product_id


### 21. Emp whose salary < department average

In [0]:
spark.sql("""
    select e.emp_id,
           e.emp_name,
           e.dept_id,
           e.salary,
           d.dept_name
    from emp_tbl e
    join (
        select dept_id, avg(salary) as avg_salary
        from emp_tbl
        group by dept_id
    ) avg_tbl
    on e.dept_id = avg_tbl.dept_id
    join dept_tbl d
    on e.dept_id = d.dept_id
    where e.salary < avg_tbl.avg_salary
""").display()


emp_id,emp_name,dept_id,salary,dept_name
1,Adam Scott,10,55000,Electronics
2,Bella Rose,20,60000,Home Appliances
4,Diana Prince,20,62000,Home Appliances
6,Fiona Glenanne,30,50000,Furniture
8,Hannah Baker,10,57000,Electronics


In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
dept_df.display()

dept_id,dept_name
10,Electronics
20,Home Appliances
30,Furniture
40,Groceries
50,Clothing


In [0]:
spark.sql("""
    select e.emp_name,
           e.salary,
           dept.dept_name
    from emp_tbl e
    join (
        select dept_id, avg(salary) as avg_salary
        from emp_tbl
        group by dept_id
    ) d
    on e.dept_id = d.dept_id
    join dept_tbl dept
    on e.dept_id = dept.dept_id
    where e.salary < d.avg_salary
""").display()

emp_name,salary,dept_name
Hannah Baker,57000,Electronics
Diana Prince,62000,Home Appliances
Fiona Glenanne,50000,Furniture
Adam Scott,55000,Electronics
Bella Rose,60000,Home Appliances


### 22. Find first and last purchase date for each customer

In [0]:
orders_df.display()

order_id,customer_id,product_id,order_date,amount
1,101,1001,2025-10-01,250.5
2,102,1002,2025-10-02,450.75
3,103,1003,2025-10-03,120.0
4,104,1002,2025-10-04,300.25
5,105,1004,2025-10-05,550.0
6,106,1005,2025-10-06,275.75
7,107,1001,2025-10-07,220.0
8,108,1003,2025-10-08,180.5


In [0]:
spark.sql("""
select customer_id,
min(order_date) as first_purchase,
max(order_date) as last_purchase
from orders_tbl
group by customer_id""").display()

customer_id,first_purchase,last_purchase
101,2025-10-01,2025-10-01
102,2025-10-02,2025-10-02
103,2025-10-03,2025-10-03
104,2025-10-04,2025-10-04
105,2025-10-05,2025-10-05
106,2025-10-06,2025-10-06
107,2025-10-07,2025-10-07
108,2025-10-08,2025-10-08


### 23. Output rows for two table join - for left/right/full/inner

### 24. Case statement